# Step 5: Spectrum and light curve from DL3

This notebook must be used with a gammapy version v0.20 or higher. Here we will use the following conda environment:

`conda activate gammapy-v1.0`

If you are running locally follow the instructions on the indico page to get the same environment: https://indico.cta-observatory.org/event/4547/

In this notebook we analyze DL3 with `gammapy` and we will produce:
1) sky map
2) the signal excess significance (on Crab data)
3) the measured Spectral Energy Distribution (SED)
4) the light curve

We will then fit the SED with a LogParabola model given by  

## $ \Phi =  \Phi_0 \left(\frac{E}{E_{ref}}\right)^{\alpha + \beta\log_{10}(E/E_{ref})}     $

where we compare it with the archival Crab spectrum, which has $\Phi_0 = 3.23\times 10^{-11}$ cm$^{-2}$ s$^{-1}$ TeV$^{-1}$, $E_{ref} = 1$ TeV, $\alpha = -2.47$, and $\beta = -0.24$.



### As usual, let's start by loading some modules (and checking the `gammapy` version)

In [ ]:
import itertools
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time
from matplotlib import pyplot as plt

In [ ]:
# We need to use gammapy v0.20 or later.

import gammapy

print(f"gammapy: v{gammapy.__version__}")

from gammapy.data import DataStore
from gammapy.datasets import Datasets, SpectrumDataset
from gammapy.estimators import FluxPointsEstimator, LightCurveEstimator
from gammapy.makers import (
    ReflectedRegionsBackgroundMaker,
    SafeMaskMaker,
    SpectrumDatasetMaker,
    WobbleRegionsFinder,
)
from gammapy.maps import Map, MapAxis, RegionGeom
from gammapy.modeling import Fit
from gammapy.modeling.models import (
    ExpCutoffPowerLawSpectralModel,
    LogParabolaSpectralModel,
    PowerLawSpectralModel,
    SkyModel,
    create_crab_spectral_model,
)
from gammapy.visualization import plot_spectrum_datasets_off_regions
from regions import CircleSkyRegion, PointSkyRegion

In [ ]:
# Customize the pyplot figure
plt.rcParams.update(
    {"figure.figsize": (12, 9), "font.size": 15, "grid.linestyle": "--"}
)

# Get the default color cycle
colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

### Load DL3 files

Here we load the DL3 files into Gammapy data containers (i.e.: **DataStore objects**) .

First we need to provide the directories where the DL3 and index files are stored.

The meaning of the following directory's labels are:  
*_wgt*: IRFs for the observation are calculated by linear interpolations of IRFs in the 3 closest test nodes. The wheigts of IRFs depends on the zenith and azimuth distance.  
*_int*: to get the "event" reconstructed values (energy, gammaness, arrival direction) the individual telescope results are weighted by their Intensities.  
*_dyn*: dynamic theta (gammaness cuts have been used as dynamic in all datasets uploaded here)  

In this example, in case we are running at the IT container, we use the data here:
`/fefs/aswg/workspace/2023_joint_analysis_school/IRF_and_DL3/input/input_step_5/DL3_wgt/`   
`/fefs/aswg/workspace/2023_joint_analysis_school/IRF_and_DL3/input/input_step_5/DL3_int/`  
produced with dynamic gammaness cuts, global theta cuts, IRF weighting and single telescope weighting.

It is possible to run this notebook on a single dataset or both. Running on both allows to check the effect of the differnt choices. If you wish to process only one set of DL3 files, please set `input_dir_1 = ""` below.

Let's start by creating DataStore objects from the input directories and take a look at one of them.

In [ ]:
#input_dir = '/fefs/aswg/workspace/2023_joint_analysis_school/IRF_and_DL3/input/input_step_5/DL3_wgt/'
#input_dir_1 = '/fefs/aswg/workspace/2023_joint_analysis_school/IRF_and_DL3/input/input_step_5/DL3_int/'
input_dir = '/home/dipierr/data/software-school-2023/IRF_and_DL3/input/input_step_5/DL3_wgt/'
input_dir_1 = '/home/dipierr/data/software-school-2023/IRF_and_DL3/input/input_step_5/DL3_int/'

double = True
if input_dir_1=='':
    double = False

data_store = DataStore.from_dir(input_dir)
if double:
    data_store_1 = DataStore.from_dir(input_dir_1)
    
# Show the observation table
data_store.obs_table

Let's take a look at the features of the observations (i.e. the runs).

In the cell below, if we set `obs_ids = None` this means that we will process all the runs. If you want to use only some of the runs, please insert instead a list of comma-separated runs.

In this step we use the Gammapy objects "Observation", which contain a lot of information (e.g.: events, hdu tables, meta data,...).

In [ ]:
obs_ids = None   # `None` means "all" observations
obs_ids_1 = None

observations = data_store.get_observations(obs_ids, required_irf="point-like")
if double:
    observations_1 = data_store_1.get_observations(obs_ids_1, required_irf="point-like")

print(observations)

### Define the ON region

Here we will collect the target position from observations metadata. To define the `on_region` we use `CircleSkyRegion` in case of global theta cuts, `PointSkyRegion` in case of dynamic theta cuts.

In [ ]:
# Get metadata from the first observation
observation = observations[0]

event_meta = observation.events.table.meta
aeff_meta = observation.aeff.meta

# Collecting the target position
target_position = SkyCoord(
    u.Quantity(event_meta["RA_OBJ"], u.deg),
    u.Quantity(event_meta["DEC_OBJ"], u.deg),
    frame="icrs",
)

if "RAD_MAX" in aeff_meta:
    # Get the global theta cut used for creating the IRFs
    on_region_radius = aeff_meta["RAD_MAX"] * u.deg
    
    # Use the circle sky region to apply the global theta cut
    on_region = CircleSkyRegion(center=target_position, radius=on_region_radius)
    
else:
    # Use the point sky region to apply dynamic theta cuts
    on_region = PointSkyRegion(target_position)
    
print("Position of the ON region: \n",on_region)

Since we are providing a second input sample, let's collect also its position since the two datasets can have different types of theta cuts (in this case however they are identical).

In [ ]:
if double:
    # Get metadata from the first observation
    observation_1 = observations_1[0]

    event_meta_1 = observation_1.events.table.meta
    aeff_meta_1 = observation_1.aeff.meta

    # Define a target position
    target_position_1 = SkyCoord(
        u.Quantity(event_meta["RA_OBJ"], u.deg),
        u.Quantity(event_meta["DEC_OBJ"], u.deg),
        frame="icrs",
    )

    if "RAD_MAX" in aeff_meta:
        # Get the global theta cut used for creating the IRFs
        on_region_radius = aeff_meta["RAD_MAX"] * u.deg

        # Use the circle sky region to apply the global theta cut
        on_region_1 = CircleSkyRegion(center=target_position, radius=on_region_radius)

    else:
        # Use the point sky region to apply dynamic theta cuts
        on_region_1 = PointSkyRegion(target_position_1)

    print("Position of the second ON region: \n",on_region_1)

### Running the data reduction chain

Here we create the energy axes (reconstructed and true energy) and we set the number of OFF regions.

We also create the Gammapy **Makers** needed to process the data.

In [ ]:
# ================
# === Settings ===
# ================

energy_min = "0.030 TeV"
energy_max = "30. TeV"
n_bins_pdec = 5

true_energy_min = "0.01 TeV"
true_energy_max = "100 TeV"
n_bins_pdec_true = 10

n_off_regions = 3

# ============
# === Main ===
# ============

energy_axis = MapAxis.from_energy_bounds(
    energy_min,
    energy_max,
    nbin=n_bins_pdec,
    per_decade=True,
    name="energy",
)

energy_axis_true = MapAxis.from_energy_bounds(
    true_energy_min,
    true_energy_max,
    nbin=n_bins_pdec_true,
    per_decade=True,
    name="energy_true",
)

#print("Energy axis:")
#print(energy_axis.edges)

on_geom = RegionGeom.create(region=on_region, axes=[energy_axis])

dataset_empty = SpectrumDataset.create(geom=on_geom, energy_axis_true=energy_axis_true)

# Create a spectrum dataset maker
dataset_maker = SpectrumDatasetMaker(
    containment_correction=False,
    selection=["counts", "exposure", "edisp"],
    use_region_center=True,
)

# Create a background maker
print(f"\nNumber of OFF regions: {n_off_regions}")

region_finder = WobbleRegionsFinder(n_off_regions=n_off_regions)
bkg_maker = ReflectedRegionsBackgroundMaker(region_finder=region_finder)

# Create a safe mask maker
safe_mask_maker = SafeMaskMaker(methods=["aeff-max"], aeff_percent=10)

Below we create new makers for the second data sample:

In [ ]:
if double:
    on_geom_1 = RegionGeom.create(region=on_region_1, axes=[energy_axis])
    dataset_empty_1 = SpectrumDataset.create(geom=on_geom_1, energy_axis_true=energy_axis_true)
    # Create a spectrum dataset maker
    dataset_maker_1 = SpectrumDatasetMaker(
        containment_correction=False,
        selection=["counts", "exposure", "edisp"],
        use_region_center=True,
    )

    # Create a background maker
    print(f"\nNumber of OFF regions: {n_off_regions}")

    region_finder_1 = WobbleRegionsFinder(n_off_regions=n_off_regions)
    bkg_maker_1 = ReflectedRegionsBackgroundMaker(region_finder=region_finder_1)

    # Create a safe mask maker
    safe_mask_maker_1 = SafeMaskMaker(methods=["aeff-max"], aeff_percent=3)

### Now let Gammapy to make the magic...

Now we apply the makers to the observations to create new datasets that contain the number of events, number of excess and background events, exposure and $\delta = 1/R_{OFF}$, where $R_{OFF}$ is the number of OFF regions. We will use this $\delta$ later to compute the Li & Ma significance.


In [ ]:
datasets = Datasets()

counts = Map.create(skydir=target_position, width=3)

# Loop over every observation
print("Running the makers...")

n_observations = len(observations)

for i_obs, observation in enumerate(observations):
    
    if (i_obs % 10) == 0:
        print(f"{i_obs}/{n_observations}")
    
    obs_id = observation.obs_id
    
    # Fill the number of events in the map
    counts.fill_events(observation.events)

    # Run the makers to the observation data
    dataset = dataset_maker.run(dataset_empty.copy(name=str(obs_id)), observation)
    dataset_on_off = bkg_maker.run(dataset, observation)
    dataset_on_off = safe_mask_maker.run(dataset_on_off, observation)

    datasets.append(dataset_on_off)

print(f"{n_observations}/{n_observations}")

# Get the information table
info_table = datasets.info_table(cumulative=True)
print(datasets.energy_ranges)

# Show the table
info_table

Again for the second sample:

In [ ]:
if double:
    datasets_1 = Datasets()

    counts_1 = Map.create(skydir=target_position_1, width=3)

    # Loop over every observation
    print("Running the makers...")

    n_observations_1 = len(observations_1)

    for i_obs, observation in enumerate(observations_1):

        if (i_obs % 10) == 0:
            print(f"{i_obs}/{n_observations_1}")

        obs_id = observation.obs_id

        # Fill the number of events in the map
        counts_1.fill_events(observation.events)

        # Run the makers to the observation data
        dataset = dataset_maker_1.run(dataset_empty_1.copy(name=str(obs_id)), observation)
        dataset_on_off = bkg_maker_1.run(dataset, observation)
        dataset_on_off = safe_mask_maker_1.run(dataset_on_off, observation)

        datasets_1.append(dataset_on_off)

    print(f"{n_observations_1}/{n_observations_1}")

    # Get the information table
    info_table_1 = datasets_1.info_table(cumulative=True)
    print(datasets_1.energy_ranges)

    # Show the table
    info_table_1

### Sky map

Let's create a counts map showing also the ON-OFF regions.
In the legend we list the run number for each set of OFF regions.

In [ ]:
plt.figure()

# Plot the count map
ax = counts.plot(add_cbar=True)

# Plot the ON position
on_geom.plot_region(ax=ax, edgecolor="red")

# Plot the OFF positions (only the first part of observations)
if n_observations < 10:
    plot_spectrum_datasets_off_regions(datasets, ax)
else:
    plot_spectrum_datasets_off_regions(datasets[0:10], ax)

ax.grid()

### Excess significance

Now let's plot the number of events and significance (i.e. sqrt(TS), in $\sigma$ units) as a function of livetime, which is the total amount of usefull time collecting data from the target, i.e. livetime $= t_{on} - t_{dead}$.

As expected, the number of excess events (and so the significance) increase faster than the background in terms of the livetime, reaching a 50 $\sigma$ significance in less than 2 hours.

In [ ]:
plt.figure(figsize=(20, 7))
grid = (1, 2)

# Plot the number of events along the livetime
plt.subplot2grid(grid, (0, 0))
plt.xlabel("Livetime [hour]")
plt.ylabel("Number of events")

plt.plot(
    info_table["livetime"].to("h"),
    info_table["excess"],
    marker="o",
    linestyle="--",
    label="Excess",
)

plt.plot(
    info_table["livetime"].to("h"),
    info_table["background"],
    marker="o",
    linestyle="--",
    label="Background",
)

plt.grid()
plt.legend()

# Plot the significance along the livetime
plt.subplot2grid(grid, (0, 1))
plt.xlabel("Livetime [hour]")
plt.ylabel("Sqrt(TS)")

plt.plot(
    info_table["livetime"].to("h"), info_table["sqrt_ts"], marker="o", linestyle="--"
)

plt.grid()

### Fitting the spectrum

We now fit the LogParabola model defined in the top of this notebook to our data. We start by creating a `SkyModel` and adding it to our datasets, we create a `Fit` that we will run on our data 

In [ ]:
source_name="Crab Nebula"

spectral_model = LogParabolaSpectralModel(
    amplitude=u.Quantity(5e-12, unit="cm-2 s-1 TeV-1"),
    alpha=2,
    beta=0.1,
    reference=u.Quantity(1, unit="TeV"),
)

sky_model = SkyModel(spectral_model=spectral_model.copy(), name=source_name)

# Add the model to the stacked dataset
stacked_dataset = datasets.stack_reduce()
stacked_dataset.models = [sky_model]

if double:    
    stacked_dataset_1 = datasets_1.stack_reduce()
    stacked_dataset_1.models = [sky_model]

# Create a fit object to run on the datasets
fit = Fit()

Here we apply the `fit` on the first sample to get the LogParabola parameters.

In [ ]:
results = fit.run(datasets=stacked_dataset)
print(results)

# Keep the best fit model
best_fit_model = stacked_dataset.models[0].spectral_model.copy()

# Show the fitted parameters
stacked_dataset.models.to_parameters_table()

Same for the second sample:

In [ ]:
if double:    
    results_1 = fit.run(datasets=stacked_dataset_1)
    print(results_1)

    # Keep the best fit model
    best_fit_model_1 = stacked_dataset_1.models[0].spectral_model.copy()

    # Show the fitted parameters
    stacked_dataset_1.models.to_parameters_table()

Now let's check if the observations are in good agreement with the predictions. We do it by plotting the number of excess events and comparing it to the number of predicted events.

In [ ]:
plt.figure()

# Plot the number of excess and predicted events
ax_spectrum, ax_residuals = stacked_dataset.plot_fit()

ax_spectrum.set_ylabel("Number of events")
ax_spectrum.grid()

plt.setp(ax_spectrum.get_xticklabels(), visible=False)
plt.subplots_adjust(hspace=0)

ax_residuals.grid()

In [ ]:
if double:
    plt.figure()

    # Plot the number of excess and predicted events
    ax_spectrum, ax_residuals = stacked_dataset_1.plot_fit()

    ax_spectrum.set_ylabel("Number of events")
    ax_spectrum.grid()

    plt.setp(ax_spectrum.get_xticklabels(), visible=False)
    plt.subplots_adjust(hspace=0)

    ax_residuals.grid()

### Estimating the flux points

We create a `FluxPointsEstimator` object to be applied on our datasets in order to evaluate the SEDs (`sed_type="e2dnde"`)

In [ ]:
# Create a flux point estimator
flux_points_estimator = FluxPointsEstimator(
    energy_edges=energy_axis.edges, source=source_name, selection_optional="all"
)

For the first data sample we have:

In [ ]:
# Run the flux point estimator to the datasets
print("Running the flux points estimator...")
flux_points = flux_points_estimator.run(datasets=stacked_dataset)

# Show the flux points table
flux_points.to_table(sed_type="e2dnde", formatted=True)

While for the second:

In [ ]:
if double:
    # Run the flux point estimator to the datasets
    print("Running the flux points estimator...")
    flux_points_1 = flux_points_estimator.run(datasets=stacked_dataset_1)

In [ ]:
flux_points_1.to_table(sed_type="e2dnde", formatted=True)

### Computing the SED

Below we show the SEDs for the two datasets in the same plot, showing the flux points and best fit models. For comparison, we plot the Crab reference SED (MAGIC, JHEAp 2015).

In [ ]:
# ================
# === Settings ===
# ================

sed_type = "e2dnde"
yunits = u.Unit("erg cm-2 s-1")

crab_model = create_crab_spectral_model("magic_lp")
#print(crab_model)
reference_models = {
    "Crab reference (MAGIC, JHEAp 2015)": crab_model,
    
}

# ============
# === Main ===
# ============

plt.figure()

energy_bounds = energy_axis.edges[[0, -1]]

lines = itertools.cycle(["--", "-.", ":"])

# Plot the flux points
ax = flux_points.plot(sed_type=sed_type, label="LST-1 + MAGIC (this tutorial), first dataset")
if double:
    ax = flux_points_1.plot(sed_type=sed_type, label="LST-1 + MAGIC (this tutorial), second dataset")


# Plot the best fit model and its error, first sample
best_fit_model.plot(
    ax=ax,
    energy_bounds=energy_bounds,
    sed_type=sed_type,
    yunits=yunits,
    color=colors[0],
    label="Best fit, first dataset"
)

best_fit_model.plot_error(
    ax=ax,
    energy_bounds=energy_bounds,
    sed_type=sed_type,
    yunits=yunits,
    facecolor=colors[0],
)

if double:
    # Plot the best fit model and its error, second sample
    best_fit_model_1.plot(
        ax=ax,
        energy_bounds=energy_bounds,
        sed_type=sed_type,
        yunits=yunits,
        color=colors[1],
        label="Best fit, second dataset "
    )

    best_fit_model_1.plot_error(
        ax=ax,
        energy_bounds=energy_bounds,
        sed_type=sed_type,
        yunits=yunits,
        facecolor=colors[1],
    )


# Plot the reference spectra
for label, model in reference_models.items():

    model.plot(
        ax=ax,
        energy_bounds=energy_bounds,
        sed_type=sed_type,
        yunits=yunits,
        label=label,
        linestyle=next(lines)
    )

ax.set_title(f"Spectral Energy Distribution of the {source_name}")

ax.set_ylim(1e-12)
ax.set_ylabel("$E^2dN/dE$ [erg cm$^{-2}$ s$^{-1}$]")
ax.grid(which="both")
ax.legend(loc="lower left")

### Computing the light curve

For the light curve, we assume that the spectral shape parameters, $\alpha$ and $\beta$, do not change over the different time bins, i.e. we freeze them at their best-fit values achieved above.

We start with the first data sample:

In [ ]:
frozen_params = ["alpha", "beta"]

sky_model = SkyModel(
    spectral_model=best_fit_model.copy(), name=source_name
)

# Freeze the spectral parameters
for param in frozen_params:
    sky_model.parameters[param].frozen = True
    
# Add the model to the datasets
datasets.models = [sky_model]

print(sky_model)


And do the same for the second:

In [ ]:
if double:
    sky_model_1 = SkyModel(
        spectral_model=best_fit_model_1.copy(), name=source_name
    )

    # Freeze the spectral parameters
    for param in frozen_params:
        sky_model_1.parameters[param].frozen = True

    # Add the model to the datasets
    datasets_1.models = [sky_model_1]

    print(sky_model_1)


Now we create a `LightCurveEstimator`.

We must set energy edges and time intervals to be used to compute the light curve. If we adopt `time_intervals=None` we will get a run-wise LC, if instead we give it a list of Astropy Time objects, we can create e.g. a daily binned LC.

In [ ]:
energy_edges = energy_axis.edges[[1,-1]]

#time_intervals = [
#    Time([59171.98, 59172.01], format="mjd", scale="utc"),
#    Time([59190.98, 59190.99], format="mjd", scale="utc"),
#    Time([59198.89, 59198.94], format="mjd", scale="utc"),
#    Time([59258.91, 59258.99], format="mjd", scale="utc"),
#    Time([59288.91, 59288.94], format="mjd", scale="utc"),
#    Time([59290.94, 59290.96], format="mjd", scale="utc"),
#]   # e.g. daily light curve 

time_intervals=None# `None` automatically makes a "run-wise" LC.

# ============
# === Main ===
# ============

# Create a light curve estimator
light_curve_estimator = LightCurveEstimator(
    energy_edges=energy_edges,
    time_intervals=time_intervals,
    source=source_name,
    selection_optional="all",
)

Now we run this estimator on the first data sample (`sed_type="flux"`)

In [ ]:
# Run the light curve estimator to the datasets
print("Running the light curve estimator...")
light_curve = light_curve_estimator.run(datasets=datasets)

# Show the light curve table
light_curve.to_table(sed_type="flux", format="lightcurve")


Now for the second data sample:

In [ ]:
if double:
    # Run the light curve estimator to the datasets
    print("Running the light curve estimator...")
    light_curve_1 = light_curve_estimator.run(datasets=datasets_1)

    # Show the light curve table
    light_curve_1.to_table(sed_type="flux", format="lightcurve")

### LC plot

Finally, we plot the LCs and compare them to the Crab reference flux.

In [ ]:
plt.figure()

lines = itertools.cycle(["--", "-.", ":"])

# Plot the light curve
ax = light_curve.plot(sed_type="flux", label="LST-1 + MAGIC (this work), first set")
if double:
    ax = light_curve_1.plot(sed_type="flux", label="LST-1 + MAGIC (this work), second set")


xlim = plt.xlim()

# Plot the reference flux
for label, model in reference_models.items():
    
    integ_flux = model.integral(energy_edges[0], energy_edges[1])
    ax.plot(xlim, np.repeat(integ_flux, 2), label=label, linestyle=next(lines))

energy_range = f"{energy_edges[0]:.3f} < $E$ < {energy_edges[1]:.1f}"

ax.set_title(f"Light curve of {source_name} ({energy_range})")
ax.set_ylabel("Flux [cm$^{-2}$ s$^{-1}$]")
ax.set_xlabel("Time [date-hour]")
ax.set_yscale("linear")
ax.legend()
ax.grid()
